In [1]:
# Imports

from split_image import split_image
import os
from math import trunc
from tqdm import tqdm
import pandas as pd
from PIL import Image
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)


## Split and I rename images

In [2]:
YEAR = '1964'

PATH = f"/scratch/project_2007251/hel{YEAR}/"
DATA_PATH = os.path.join(PATH, 'colored_images/')
RESULT_PATH = os.path.join(PATH, 'small_images/')
IMAGE_FOLDER_NAME = f"hel{YEAR}"
CSV_PATH = os.path.join(PATH, f'avoindata:Ortoilmakuva_{YEAR}_range_info.csv')
CSV_RESULT_PATH = os.path.join(PATH, 'output.csv')

if not os.path.exists(RESULT_PATH):
    os.makedirs(RESULT_PATH)
else:
    print('There is a folder exists.')
    print(f'There are {len(os.listdir(RESULT_PATH))} files.')

In [3]:
# fix the bug if run multiple times. :(

# for image_name in tqdm(os.listdir(DATA_PATH)):
#     if image_name.endswith(".png"):  # check if the file is a PNG image
#         # Split the filename into its parts
#         name, ext = os.path.splitext(image_name)
#         year, part1, part2 = name.split("_")
#         part1 = part1.replace("00", "0")
#         part2 = part2.replace("00", "0")

#         new_name = f"{year}_{part1}_{part2}{ext}"
#         os.rename(os.path.join(DATA_PATH, image_name), os.path.join(DATA_PATH, new_name))
#         # print(f"Renamed {image_name} to {new_name}")
#         # break


In [10]:
# Rename unsplit images according to new coordinate system
# Multiplies each coordinate by one to make room for splitting each image to 10x10 smaller images
for image_name in os.listdir(DATA_PATH):
    
    # Create new name for the image with correct coordinate
    coordinates = image_name.split(".")[0].split("_")[1:]
    new_image_name = IMAGE_FOLDER_NAME + "_" + str(int(coordinates[0])*10) + "_" + str(int(coordinates[1])*10) + ".png"
    
    # Rename file
    old_img_path = DATA_PATH + image_name
    new_img_path = DATA_PATH + new_image_name
    
    os.rename(old_img_path, new_img_path)

    
# Fix names with just one "0" in their name -> replace with "00"
for image_name in os.listdir(DATA_PATH):
    
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        
        start, old_x, old_y = image_name.split(".")[0].split("_")
        new_x = old_x
        new_y = old_y
        
        if new_x == "0" or new_y == "0":

            if new_x == "0":
                new_x = "00"
            if new_y == "0":
                new_y = "00"

            new_image_name = start + "_" + new_x + "_" + new_y + ".png"

            # Rename file
            old_img_path = DATA_PATH + image_name
            new_img_path = DATA_PATH + new_image_name

            os.rename(old_img_path, new_img_path)

In [4]:
# Split images

print(DATA_PATH)

for image_name in tqdm(os.listdir(DATA_PATH)):
    
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        
        img_path = DATA_PATH + image_name
        split_image(image_path=img_path,
                    rows=10,
                    cols=10,
                    should_square=False,
                    should_quiet=True,
                    should_cleanup=False,
                    output_dir=RESULT_PATH)

/scratch/project_2007251/hel1964/colored_images/


100%|██████████| 504/504 [1:30:10<00:00, 10.74s/it]


In [5]:
# import cv2
# print(RESULT_PATH)
# list_5120 = []
# for i in tqdm(os.listdir(RESULT_PATH)):
#     # print(i)
#     img = cv2.imread(os.path.join(RESULT_PATH,i))
#     if img.shape != (512, 512, 3):
#         print(o, img.shape)
#         list_5120.append(fileList[i])
#         # break

In [6]:
# Rename newly split images according to new coordinate system
# Fix naming convention to match our coordinate system
print(RESULT_PATH)

for image_name in tqdm(os.listdir(RESULT_PATH)):
    
    # Only modify images
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        
        # Get x an y value from old name
        index_number = int(image_name.split(".")[0].split("_")[-1])
        x = index_number % 10
        y = 9 - trunc(index_number / 10)
        
        # Split old name into parts
        start, old_x, old_y, i = image_name.split(".")[0].split("_")
        
        # Build new name from parts and new manipulated coordinates
        new_y = old_y[:-1] + str(y)
        new_x = old_x[:-1] + str(x)
        new_image_name = start + "_" + new_x + "_" + new_y + ".png"

        # Rename file
        old_img_path = RESULT_PATH + image_name
        new_img_path = RESULT_PATH + new_image_name
        os.rename(old_img_path, new_img_path)

/scratch/project_2007251/hel1964/small_images/


100%|██████████| 50400/50400 [00:17<00:00, 2845.65it/s]


## Create csv file for splitted images

In [7]:
# Create info csv files for splitted data
print(CSV_RESULT_PATH)

x_min = 24.8121
y_min = 60.1415
x_max = 25.1984
y_max = 60.3033

cols = {'Location': [],
        'x_min': [],
        'y_min': [],
        'x_max': [],
        'y_max': []}
csv_df = pd.DataFrame(cols)

for x in tqdm(range(0,420)):
    for y in range(0,120):
        
        location_x = str(x)
        location_y = str(y)
        if x < 10:
            location_x = "0" + location_x
        if y < 10:
            location_y = "0" + location_y
        
        location = "hel2019_" + location_x + "_" + location_y + ".png"
        
        csv_df = csv_df.append(
        {'Location': location,
            'x_min': x_min + x * 0.0009001,
            'x_max': x_min + x * 0.0009001 + 0.0009001,
            'y_min': y_min + y * 0.001272793,
            'y_max': y_min + y * 0.001272793 + 0.001272793
        },
        ignore_index = True)
        
csv_df.to_csv(CSV_RESULT_PATH, header=False, index=False)

/scratch/project_2007251/hel1964/output.csv


100%|██████████| 420/420 [01:08<00:00,  6.15it/s]


## Combine splitted images back into one

In [ ]:
IMAGE_FOLDER_PATH = '/Users/hartih/Documents/GitHub/ForestPatches/data/hel2015_sample/images/'
IMAGE_NAME_BEGINNING = 'hel2015_'
EXAMPLE_IMAGE_URL = '/Users/hartih/Documents/GitHub/ForestPatches/data/hel2015_sample/images/hel2015_4_9.png'
TARGET_IMAGE = '/Users/hartih/Downloads/test_combination_png.png'

# Get list of image names to be combined
image_names = os.listdir(IMAGE_FOLDER_PATH)

# Filter out non-image files (e.g. DS_store)
def filter_images(image_name):
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
          return True
    return False
image_names =  filter(filter_images, image_names)

image_names = [name.split(".")[0].split("_") for name in image_names]
x_list = [int(i[1]) for i in image_names]
y_list = [int(i[2]) for i in image_names]

# Create sorted list of xs
x_list = list(dict.fromkeys(x_list))
x_list.sort()

# Create reverse sorted list of ys
y_list = list(dict.fromkeys(y_list))
y_list.sort()
y_list.reverse()

# Combine images into one big one
example_image = Image.open(EXAMPLE_IMAGE_URL)
dst = Image.new('RGB', (example_image.width * len(x_list), example_image.height * len(y_list)))
i_x = 0
for x in x_list:
    i_y = 0
    for y in y_list:
        
        # Make sure x and y in name matches splitted images
        str_x = str(x)
        str_y = str(y)
        #if len(str_x) == 1:
        #    str_x = "0" + str_x
        #if len(str_y) == 1:
        #    str_y = "0" + str_y
        
        # Paste image to new combined image
        image_name = IMAGE_NAME_BEGINNING + str_x + "_" + str_y + ".png"
        print(image_name)
        image_url = IMAGE_FOLDER_PATH + image_name
        image = Image.open(image_url)
        print(image_url)

        dst.paste(image, (i_x * example_image.width, i_y * example_image.height))
        i_y += 1
    i_x += 1

# Save combined image
dst.save(TARGET_IMAGE)